In [1]:
from requests import get
from bs4 import BeautifulSoup
from random import choice
from csv import DictReader, DictWriter

In [2]:
class Celebrity:
    def __init__(self, name, born = ''):
        self.name = name
        self.born = born
        self.quotes = []
    
    def __str__(self):
         return f"{self.name}: {self.born}"
    
    def display_born(self):
        return f"The author was born {self.born}"
    
    def add_quote(self, quote):
        self.quotes.append(quote)
    
    def get_quote(self):
        if len(self.quotes) > 1:
            return choice(self.quotes)
        return self.quotes[0]
    
    def initials(self):
        """
        return the celebrity's first and last name initials, each wrapped in a message (a hint)
        """
        
        name = self.name.split(" ")
        fname_initial = name[0][0]
        
        if len(name) == 2 or self.name.endswith('fils'):
            lname_initial = name[1][0]
        elif len(name) == 3 or (len(name) == 4 and self.name.endswith('.jr')):
            lname_initial = name[2][0]
        
        return (f"The author's first name starts with {fname_initial}", f"The author's last name starts with {lname_initial}")

In [3]:
def request(endpoint):
    if type(endpoint) == int:
        url = f"https://quotes.toscrape.com/page/{endpoint}"
    else:
        url = f"https://quotes.toscrape.com{endpoint}"
    
    response = get(url)
    return BeautifulSoup(response.text, "html.parser")

In [4]:
def authors(request):
    authors = set()
    page = 1
    
    while True:
        soup = request(page)
        
        for author in soup.select(".author"):
            authors.add(author.text)
        
        if not soup.find(class_ = 'next'):
            break
        page += 1
    return authors

In [5]:
authors = authors(request)

In [6]:
def get_quotes(request):
    quotes = []
    page = 1
    
    while True:
        soup = request(page)
        quotes.extend(soup.select(".quote"))
        
        if not soup.find(class_ = 'next'):
            break
        page += 1
        
    return quotes

In [7]:
quotes = get_quotes(request)

In [8]:
def celeb_instances(request, authors, quotes):
    celebs = []

    for author in authors:
        celeb = Celebrity(author)
        celebs.append(celeb)
        
        for quote in quotes:
            if quote.find(class_ = 'author').text == author:
                quotation = quote.find(class_ = 'text').text
                celeb.add_quote(quotation)
                
                if not celeb.born:
                    about = quote.find("a")['href']
                    soup = request(about)
                    birth_date, birth_location = soup.find(class_ = 'author-details').select('span')
                    born = f"{birth_date.get_text()} {birth_location.get_text()}"
                    celeb.born = born
    return celebs

In [9]:
celebs = celeb_instances(request, authors, quotes)

In [10]:
def hint(guesses, **kwargs):
    if guesses == 3:
        hint = kwargs["born"]
    elif guesses == 2:
        hint = kwargs["fname_initial"]
    else:
        hint = kwargs["lname_initial"]
    return hint

In [16]:
def game():
    celebrity = choice(celebs)
    born = celebrity.display_born()
    quote = celebrity.get_quote()
    fname_initial, lname_initial = celebrity.initials()
    game_on = True
    
    while game_on:
        print("Here's a quote:")
        print(celebrity.get_quote())
        
        guesses = 4
        guess = ''
        
        while guesses > 0:
            guess = input(f"\nWho said this? Guesses remaining: {guesses}: ")
            
            if guess == celebrity.name:
                print("Congrats! You've guessed correctly.")
                play_again = input("Play again? (y/n) ")
                if play_again.lower() == 'y':
                    print("Here we go again.")
                    print("")
                    game()
                else:
                    print("Catch you next time.")
                    return
            else:
                guesses -= 1
                print(hint(guesses, born = born, fname_initial = fname_initial, lname_initial = lname_initial))
        else:
            print("\nYou've exhausted all 4 guesses.")
            print(f"The quote's author is {celebrity.name}")
            play_again = input("Play again? (y/n) ")
            if play_again.lower() == 'y':
                print("Here we go again.")
                print("")
                game()
            else:
                print("Catch you next time.")
                return

In [ ]:
game()